# Model Assessment with Ship Data
**Before you begin.**
You will edit this Colab notebook by adding/changing code and adding text to answer questions and provide explanations and insights.  When you are finished, you can print to PDF in order to easily generate a report.  To earn full points,  
*   try your best to avoid code that extends past the vertical bar in code cells.  It will not print well.
*   print text along with the answer/explanation, if answering/explaining with code.  For example, use `print(f'The bias is {bias}.')` instead of `print(bias)`.  This way, your instructor knows that you know how to interpret what you are seeing,
*   use italicized text if answering/explaining with text.  This will help your answers stand out, and
*   identify and explain some key observations that were not explcitily asked for in this assignment.  Sometimes the instructor wants to see if you know what to look for on your own.

You can add cells of either text or code by hovering over the horizontal edge of the cell just before where you want to add a new cell.  Click either "+ text" or "+ code", as appopriate.  You may have to click in the previous cell in order to see "+ text" or "+ code"


---



**Assignment summary.** This assignment will have you rigorously evaluate a logistic regression model trained to identify ships in the [San Francisco ship aerial imagery dataset](https://www.kaggle.com/datasets/rhammell/ships-in-satellite-imagery).  In doing so, you will split the dataset into training and testing partitions and observe the accuracy, precision, recall, F1-score, confusion matrix and Matthews correlation coefficient on both partitions, using black-box functions to assist you.  You will learn insights regarding class-imbalance and the pros and cons of each performance metric.  Note the images have been converted to black and white for ease of processing.  The dataset is the same as in the previous assignment.

---


**Import the data.** Connect this Colab notebook to your Google Drive.  This code is complete.

In [ ]:
# Given your permission, this will connect your notebook to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Load in the ship data from Google Drive and print out the number of images.  The name of the folder containing all the ships should be `shipdata_MLcourse`, otherwise you need to change the code below accordingly.  The variable `img_data` contains all of the pixel information for each of the images.  This code is complete.

In [ ]:
# Now the ship data can be loaded into the notebook (presuming you uploaded..
# ...the data previously into your Google Drive)
import os, pickle

my_path = '/content/drive/My Drive/shipdata_MLcourse'
with open(os.path.join(my_path, 'img_data.pickle'), 'rb') as handle:
  img_data = pickle.load(handle)

print(f'The number of images is {len(img_data)}.') # print the number of images
print(f'Each image is a vector of length {len(img_data[0])}.')
print(f'The shape of img_data is {img_data.shape}.')          

Get the targets vector `y`: 1 if the image contains a ship, 0 for no ship (or partial ship). This code is complete.


In [ ]:
with open(os.path.join(my_path, 'targets.pickle'), 'rb') as handle:
  y = pickle.load(handle)
  print(f'The length of y is {len(y)}.')

Check out some of the ship images!  Each image is a vector of length 6400 that can be reshaped into 80 by 80 to make a small picture.  Change the value of `my_index` to view different images.  Remember that Python indexing starts at zero.  The code will print out the target as well.  Images towards the end of the dataset (larger indices) contain the ships. The code is complete.

In [ ]:
from google.colab.patches import cv2_imshow

my_index = 1000        # change this number
img = img_data[my_index].reshape((80,80))
cv2_imshow(img)  # display image

if y[my_index] == 0:
  print('This image is no ship or partial ship.')
else:
  print('This image is ship.')

**Train-test a logistic regression model with dimensionally reduced data.** Each image is 80 x 80 = 6400 pixels, which, for reasons you will understand later in the course, is too large for logistic regression and other classification algorithms. Instead you will work with vectors of length 300 that well describe the images. We call this a "dimesionally reduced" dataset, since each image is now encoded with 300 numbers instead of 6400 numbers. How exactly these 300-length vectors come to be is from a technique you will learn later in the course (called principal component analysis).

Load in the dimensionally reduced dataset.  The code below is complete.

In [ ]:
with open(os.path.join(my_path, 'X_pca.pickle'), 'rb') as handle:
  X = pickle.load(handle)
  print(f'The shape of X is {X.shape}.')

The classes are imbalanced.  Let's count how many examples of each class are contained in the entire dataset.  Use the double-equals operator on `y` to perform a logical test.  The `True`/`False` that it returns can be interpreted as `1`/`0`.  Thus, you may use the `sum` function from the `numpy` library to count how many are `True`.  How many examples are in each class?

In [ ]:
import numpy as np
num_noship =    # TODO: Use == with the sum function to count noship
num_ship =      # TODO: Use == with the sum function to count ship
print(f'There are {num_noship} examples of the no-ship class.')
print(f'There {num_ship} examples of the ship class.')

Use [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to randomly partition the dimensionally reduced dataset into training and testing partitions.  Set the function to split 75% train and 25% test.  Use a `random_state` of 0.  Then, use [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to train the model on the training set and its `predict` method to make predictions on the training set and, separately, the testing set.  These should be named `ytrn_hat` and `ytst_hat`, respectively.  You may find that you have to increase the maximum number of iterations in order for `LogisticRegression` to converge.

Also, recall that the data should be normalized to zero mean and unit variance.  To do this amidst the partitioning, you should normalize the data after the split.  First normalize the training set, then normalize the testing set with the same parameters.  Use [StandarScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and its attributes and methods `mean_`, `variance_`, `fit_transform` and `transform` to help you.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
# TODO: Randomly partition the data into training and testing sets...
# ...use train_test_split.


# TODO: Normalize (transform) the training set.  Use fit_transform 


# TODO: Normalize (transform) the testing set.  Use transform 


# TODO: Train a logistic regression model with the normalized training data...
# ...use LogisticRegression.  Then, use the model to predict the normalized...
# ...training data and, separately, normalized test data

ytrn_hat = 
ytst_hat = 

print(f'{len(ytrn_hat)} predictions made in the training set.')
print(f'{len(ytst_hat)} predictions made in the test set.')

How good are the predictions?  Now you will answer this question by computing the following metrics on both the training and testing sets:
*   accuracy as a percentage
*   confusion matrix
*   precision
*   recall
*   F1-score
*   Matthews correlation coefficient

For accuracy, you may use the double-equals operator and the `sum` function, similar to a previous cell, to count the number of times the predictions match the actual targets.  For other metrics, use the functions [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) and [matthews_corrcoef](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html) to help you.  It is recommended you compute the confusion matrix first, and then use it to calculate precision, recall, F1-score.  

What do you notice?  When you are finished, change `random_state` in the previous cell (which will randomly split the dataset differently), run the cell, and then run the current cell again.  Does your conclusion(s) still hold?

Why is accuracy not good enough to measure the performance?


In [ ]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef

# TODO: Determine and observe the training/testing accuracy


# TODO: Determine and observe the training/testing confusion matrix


# TODO: Determine and observe the training/testing precision, recall, F1-score


# TODO: Determine and observe the training/testing MCC


You looked at the algorithm performance for both the training and testing sets.  This was for educational purposes only; in practice, we do not typically report the performance on the training set.  Why not?

Below, you will use [ConfusionMatrixDisplay](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html) to display the confusion matrix of only the test set.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
# TODO: Display the confusion matrix of the test set.  
# Use the from_predictions method


One issue with your work so far is that you only split the dataset once.  One paritcularly anomalous testing set could sabotage the results, same with an amonalous training set.  When training is quick as in this project, best practice is to use cross-validation to randomly split the data K times in a way that each example is used in the test set exactly one time.  That way, we obtain K performance metrics and one single split does not sabotage the results.  Below, you will use K = 5 cross-validation and, to keep the programming simple, only measure the accuracy each of the five times the model is train-tested.  The code below employs a pipeline for the sequence: split -> normalize -> train-test.  A pipeline is beyond the scope of the course.  Finish the code so that [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) uses 5 splits.  At the end of the code, use the `mean` and `std` methods to calculate the mean and deviation of `accuracies` across the 5 splits. As opposed to a single split, this is a better estimate of the performance of your model.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
clf = LogisticRegression()
pipeline = Pipeline([('transformer', scaler), ('estimator', clf)])
cv = KFold(n_splits=5)  #TODO: split the data five times
accuracies = cross_val_score(pipeline, X, y, cv = cv)

print(accuracies)
# TODO: Calculate and print the mean and deviation of accuracies


This is the end of what is required.  You can easily generate a report by navigating to File -> Print and then selecting to save to PDF.  You will turn in this PDF.  Be sure that the report shows your code's output (that is, you've run each cell), and your notes/explanations are italicized.  Continue on for extra insights.

**Go further.** Regularization is a tool in ML that can help prevent overfitting and can be controlled with the `C` parameter in [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).  Now you will investigate the role of regularization in train/test accuracies.  

(Much of the "TODO" code can be copied from previous cells.)  Begin by randomly splitting the dataset into training and testing; only one split is fine; use `train_test_split`.  Use `StandardScaler` to normalize the training data, and then normalize the test data with the same parameters.  The code below creates an array of `C` values of  100 logarithmically equally spaced values beginning at 0.001 and ending at 100.  Then, the code loops through each one of these values.  Inside the loop, instantiate `LogisticRegression` with regularization, then train/test and calculate the train and test accuracies as percentages.  Append the results to the respective (intitially empty) lists `accuracy_trn` and `accuracy_tst`.  Outside of the loop, the code will plot the results.  Describe what you see.

Does a large `C` imply a lot or little regularization?  Read the documentation about `LogisticRegression`.

In [ ]:
import matplotlib.pyplot as plt
# TODO: Randomly partition the data into training and testing sets...
# ...use train_test_split.

# TODO: Normalize (transform) the training set.  Use fit_transform 

# TODO: Normalize (transform) the testing set.  Use transform 


Cvalues = np.logspace(-3,2,100)
accuracy_trn = []
accuracy_tst = []
for C in Cvalues:
  # TODO: Train a logistic regression model with the normalized training data...
  # ...use LogisticRegression.  Then, use the model to predict the normalized...
  # ...training data and, separately, normalized test data

  
  accuracy_trn.append(
  accuracy_tst.append(

plt.semilogx(Cvalues,accuracy_trn)
plt.semilogx(Cvalues, accuracy_tst)
plt.legend(('training', 'testing'))
plt.title('Logistic Regression Accuracy')
plt.ylabel('accuracy (%)')
plt.xlabel('C value')
plt.grid()